## Extracting 2020 movies from Wikipedia

In [1]:
import pandas as pd
import numpy as np
from tmdbv3api import TMDb
import json
import requests
import requests
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\pradi\AppData\Local\Temp\ipykernel_18968\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN
...,...,...,...,...,...,...,...
269,DECEMBER,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,NaN,[240]
270,DECEMBER,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[241]
271,DECEMBER,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[242]
272,DECEMBER,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[243]


In [10]:
tmdb = TMDb()
tmdb.api_key='e2f23b720ffb3e476ae26f11f7ab66fd'

In [11]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [12]:
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

In [13]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.,genres
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN,Horror Mystery Thriller
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN,Action Horror Science Fiction Thriller
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN,Comedy
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN,Drama
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN,Crime Thriller Drama
...,...,...,...,...,...,...,...,...
269,DECEMBER,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,NaN,[240],Action Fantasy Family Comedy
270,DECEMBER,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[241],Drama Western Adventure Action
271,DECEMBER,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[242],Drama
272,DECEMBER,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[243],Thriller Crime Drama


In [14]:
df_2020_movies = df[['Title', 'Cast and crew', 'genres']]

In [15]:
df_2020_movies

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama
...,...,...,...
269,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Action Fantasy Family Comedy
270,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure Action
271,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama
272,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama


In [16]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [17]:
df_2020_movies['director_name'] = df_2020_movies['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\pradi\AppData\Local\Temp\ipykernel_18968\3920122266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020_movies['director_name'] = df_2020_movies['Cast and crew'].map(lambda x: get_director(str(x)))


In [18]:
def get_actor_1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])


In [19]:
df_2020_movies['actor_1_name'] =df_2020_movies['Cast and crew'].map(lambda x: get_actor_1(str(x)))

C:\Users\pradi\AppData\Local\Temp\ipykernel_18968\1739264240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020_movies['actor_1_name'] =df_2020_movies['Cast and crew'].map(lambda x: get_actor_1(str(x)))


In [20]:
def get_actor_2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [21]:
df_2020_movies['actor_2_name'] = df_2020_movies['Cast and crew'].map(lambda x: get_actor_2(str(x)))

C:\Users\pradi\AppData\Local\Temp\ipykernel_18968\564821508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020_movies['actor_2_name'] = df_2020_movies['Cast and crew'].map(lambda x: get_actor_2(str(x)))


In [22]:
def get_actor_3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [23]:
df_2020_movies['actor_3_name'] = df_2020_movies['Cast and crew'].map(lambda x: get_actor_3(str(x)))

C:\Users\pradi\AppData\Local\Temp\ipykernel_18968\3497776232.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020_movies['actor_3_name'] = df_2020_movies['Cast and crew'].map(lambda x: get_actor_3(str(x)))


In [24]:
df_2020_movies

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
...,...,...,...,...,...,...,...
269,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Action Fantasy Family Comedy,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin
270,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure Action,Paul Greengrass,Tom Hanks,Helena Zengel,NaN
271,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
272,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie


In [25]:
df_2020_movies = df_2020_movies.rename(columns={'Title':'movie_title'})

In [26]:
new_2020_movie_list = df_2020_movies.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [27]:
new_2020_movie_list

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,Three Christs
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,Inherit the Viper
...,...,...,...,...,...,...
269,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,We Can Be Heroes
270,Paul Greengrass,Tom Hanks,Helena Zengel,NaN,Drama Western Adventure Action,News of the World
271,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,One Night in Miami...
272,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,Promising Young Woman


In [28]:
new_2020_movie_list['actor_2_name'] = new_2020_movie_list['actor_2_name'].replace(np.nan, 'unknown')
new_2020_movie_list['actor_3_name'] = new_2020_movie_list['actor_3_name'].replace(np.nan, 'unknown')

In [29]:
new_2020_movie_list['movie_title'] = new_2020_movie_list['movie_title'].str.lower()

In [30]:
new_2020_movie_list['combine'] = new_2020_movie_list['actor_1_name']+' '+new_2020_movie_list['actor_2_name']+' ' + \
    new_2020_movie_list['actor_3_name']+' ' + \
    new_2020_movie_list['director_name']+' '+new_2020_movie_list['genres']


In [31]:
new_2020_movie_list

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,combine
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,three christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
269,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
270,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,Drama Western Adventure Action,news of the world,Tom Hanks Helena Zengel unknown Paul Greengras...
271,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
272,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


## Extracting 2021 movies from Wikipedia

In [32]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [33]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\pradi\AppData\Local\Temp\ipykernel_18968\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [34]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,1.0,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2],NaN
1,JANUARY,13.0,The White Tiger,Netflix,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN,NaN
2,JANUARY,14.0,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[3],NaN
3,JANUARY,15.0,The Dig,Netflix / Clerkenwell Films,Simon Stone (director); Moira Buffini (screenp...,[4],NaN
4,JANUARY,15.0,Outside the Wire,Netflix,"Mikael Håfström (director); Rob Yescombe, Rowa...",[5],NaN
...,...,...,...,...,...,...,...
353,DECEMBER,25.0,The Tragedy of Macbeth,Apple TV+ / A24 / IAC Films,Joel Coen (director/screenplay); Denzel Washin...,NaN,[271]
354,DECEMBER,25.0,A Journal for Jordan,Columbia Pictures / Escape Artists / Bron Studios,Denzel Washington (director); Virgil Williams ...,NaN,[272]
355,DECEMBER,25.0,American Underdog,Lionsgate,"Erwin brothers (directors); Jon Erwin, David A...",NaN,[273]
356,DECEMBER,26.0,Memoria,Neon,Apichatpong Weerasethakul (director/acreenplay...,NaN,[274]


In [35]:
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

In [36]:
df_2021_movies = df[['Title','Cast and crew','genres']]

In [37]:
df_2021_movies

,Title,Cast and crew,genres
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,Horror Action War
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Drama
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Comedy Crime Drama
3,The Dig,Simon Stone (director); Moira Buffini (screenp...,Drama History
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa...",Thriller Action Science Fiction
...,...,...,...
353,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...,Drama War
354,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...,Drama Romance
355,American Underdog,"Erwin brothers (directors); Jon Erwin, David A...",Drama
356,Memoria,Apichatpong Weerasethakul (director/acreenplay...,Drama Science Fiction Mystery


In [38]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [39]:
df_2021_movies['director_name'] = df_2021_movies['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\pradi\AppData\Local\Temp\ipykernel_18968\465748157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021_movies['director_name'] = df_2021_movies['Cast and crew'].map(lambda x: get_director(str(x)))


In [40]:
def get_actor_1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [41]:
df_2021_movies['actor_1_name'] = df_2021_movies['Cast and crew'].map(lambda x: get_actor_1(str(x)))

C:\Users\pradi\AppData\Local\Temp\ipykernel_18968\263207904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021_movies['actor_1_name'] = df_2021_movies['Cast and crew'].map(lambda x: get_actor_1(str(x)))


In [42]:
def get_actor_2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [43]:
df_2021_movies['actor_2_name'] = df_2021_movies['Cast and crew'].map(lambda x: get_actor_2(str(x)))

C:\Users\pradi\AppData\Local\Temp\ipykernel_18968\1703673716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021_movies['actor_2_name'] = df_2021_movies['Cast and crew'].map(lambda x: get_actor_2(str(x)))


In [44]:
def get_actor_3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [45]:
df_2021_movies['actor_3_name'] = df_2021_movies['Cast and crew'].map(lambda x: get_actor_3(str(x)))

C:\Users\pradi\AppData\Local\Temp\ipykernel_18968\972348692.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021_movies['actor_3_name'] = df_2021_movies['Cast and crew'].map(lambda x: get_actor_3(str(x)))


In [46]:
df_2021_movies

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,Horror Action War,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Drama,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Comedy Crime Drama,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant
3,The Dig,Simon Stone (director); Moira Buffini (screenp...,Drama History,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa...",Thriller Action Science Fiction,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham
...,...,...,...,...,...,...,...
353,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...,Drama War,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel
354,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...,Drama Romance,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian
355,American Underdog,"Erwin brothers (directors); Jon Erwin, David A...",Drama,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid
356,Memoria,Apichatpong Weerasethakul (director/acreenplay...,Drama Science Fiction Mystery,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar


In [47]:
df_2021_movies = df_2021_movies.rename(columns={'Title':'movie_title'})

In [48]:
new_2021_movie_list = df_2021_movies.loc[:, [
    'director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'genres', 'movie_title']]

In [49]:
new_2021_movie_list['actor_2_name'] = new_2021_movie_list['actor_2_name'].replace(np.nan, 'unknown')
new_2021_movie_list['actor_3_name'] = new_2021_movie_list['actor_3_name'].replace(np.nan, 'unknown')

In [50]:
new_2021_movie_list['movie_title'] = new_2021_movie_list['movie_title'].str.lower()

In [51]:
new_2021_movie_list['combine'] = new_2021_movie_list['actor_1_name'] + ' ' + new_2021_movie_list['actor_2_name'] + ' '+ new_2021_movie_list['actor_3_name'] + ' '+ new_2021_movie_list['director_name'] +' ' + new_2021_movie_list['genres']

In [52]:
new_2021_movie_list

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,combine
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,shadow in the cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,the white tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,locked down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,the dig,Carey Mulligan Ralph Fiennes Lily James Simon ...
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,outside the wire,Anthony Mackie Damson Idris Emily Beecham Mika...
...,...,...,...,...,...,...,...
353,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
354,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
355,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...
356,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar,Drama Science Fiction Mystery,memoria,Apichatpong Weerasethakul (director/acreenplay...


In [53]:
combine_df = new_2020_movie_list.append(new_2021_movie_list, ignore_index=True)

C:\Users\pradi\AppData\Local\Temp\ipykernel_18968\4053330386.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combine_df = new_2020_movie_list.append(new_2021_movie_list, ignore_index=True)


In [54]:
combine_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,combine
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,three christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
627,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
628,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
629,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...
630,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar,Drama Science Fiction Mystery,memoria,Apichatpong Weerasethakul (director/acreenplay...


In [55]:
old_processing_data = pd.read_csv('../Data/ProcessedData/wikipedia2018_19MoviesPreprocessData.csv')

In [56]:
old_processing_data

,Unnamed: 0,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,combine
0,0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...,...
5873,5873,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,spies in disguise,Will Smith Tom Holland Rashida Jones Nick Brun...
5874,5874,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5875,5875,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action Thriller History,1917,George MacKay Dean-Charles Chapman Mark Strong...
5876,5876,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [71]:
new_final_df = old_processing_data.append(combine_df, ignore_index=True)

C:\Users\pradi\AppData\Local\Temp\ipykernel_18968\3019476396.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_final_df = old_processing_data.append(combine_df, ignore_index=True)


In [72]:
new_final_df

,Unnamed: 0,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,combine
0,0.0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,1.0,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,2.0,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,3.0,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,4.0,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...,...
6505,NaN,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
6506,NaN,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
6507,NaN,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...
6508,NaN,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar,Drama Science Fiction Mystery,memoria,Apichatpong Weerasethakul (director/acreenplay...


In [73]:
new_final_df.isna().sum()

Unnamed: 0       632
director_name      0
actor_1_name       0
actor_2_name       0
actor_3_name       0
genres             5
movie_title        1
combine            5
dtype: int64

In [74]:
new_final_df = new_final_df.drop('Unnamed: 0', axis=1)

In [75]:
new_final_df = new_final_df.dropna(how='any')

In [76]:
new_final_df.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
combine          0
dtype: int64

In [77]:
new_final_df.to_csv('../Data/ProcessedData/wikipedia2020_21MoviesProcessing.csv')

In [78]:
new_final_df.shape

(6504, 7)